In [2]:
import csv
from math import sqrt
from math import log2
from collections import Counter

In [3]:
# 讀取 txt 檔案
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        data = [row for row in reader]
    return data
# 讀取資料
data = load_data('glass.txt')

In [4]:
# 將原始 list of list 型態的 data 轉換為 dict of list
def table_to_column_dict(data, columns, convert_numeric = True):
    # 初始化一個空的 dictionary，key 是 attributes 名稱，value 是空 list：
    df_dict = {col: [] for col in columns}
    # 對每一列資料逐欄掃描，同時把欄位名稱 (col) 跟對應值 (val) 配對起來
    for row in data:
        for col, val in zip(columns, row):
            if convert_numeric:
                try:
                    val = float(val)
                except ValueError:
                    pass  # 若轉不了 float，就保持原樣（例如 Id）
            df_dict[col].append(val)
    return df_dict
# 定義欄位名稱
columns = ["Id","RI","Na","Mg","Al","Si","K","Ca","Ba","Fe","class"]
df = table_to_column_dict(data,columns)

In [5]:
X = columns.copy()
X.remove("Id")
X.remove("class")
y = 'class'

### Feature Selection 函式

In [353]:
# 計算 feature entropy
def entropy(df,feature):  
    att_value = df[feature]  # 取出 dict 的特定 attribute 的所有資料
    value_count = Counter(att_value)          # 計算所有可能值的個數
    total = len(att_value)
    prob = [count / total for key,count in value_count.items()]  # 計算每個 attribute_value 的機率
    return -sum(p * log2(p) for p in prob)

# 計算特徵 X、Y 間的 Mutual Information
def mutual_information(df,X, Y):
    X_list = df[X]
    Y_list = df[Y]
    # 計算 X 和 Y 的熵
    H_X = entropy(df ,X)
    H_Y = entropy(df ,Y)
    # 計算 X 和 Y 的聯合機率
    joint_pairs = list(zip(X_list, Y_list))
    joint_counts = Counter(joint_pairs)
    total = len(X_list)
    joint_prob = [count / total for key,count in joint_counts.items()]
    H_X_Y = -sum(p * log2(p) for p in joint_prob)
    return H_X + H_Y - H_X_Y
    
# 計算特徵 X、Y 的 symmetric uncertainty
def cal_su(df,X,Y):
    H_X = entropy(df,X)
    H_Y = entropy(df,Y)
    if H_X == 0 and H_Y == 0:
        return 0
    return 2 * (mutual_information(df,X,Y) / (H_X + H_Y))

# 計算選取的特徵子集對於類別預測的 Goodness
def Goodness(df,feature_subset,label):
    su_X_C = 0
    sum_su_X_Y = 0  
    # 計算 feature_subset 內所有特徵對於類別值的 Symmetric uncertainty
    su_X_C = sum(cal_su(df,X,label) for X in feature_subset)
    
    # 計算 feature_subset 內所有兩兩特徵間的 Symmetric uncertainty
    for feature_i in feature_subset:
        for feature_j in feature_subset:
            sum_su_X_Y += cal_su(df,feature_i,feature_j)
    if sum_su_X_Y == 0:
        return 0
    return su_X_C / sqrt(sum_su_X_Y)

def forward_selection(df, X, y):
    select_features = []    
    best_score = 0.0        
    remaining_features = X.copy()  
    # 持續檢查直到沒有可以選擇的 feature
    i = 1   
    while(len(remaining_features) > 0):
        scores = []  
        for feature in remaining_features:
            # temp_features 暫存此次循環的特徵組合 => 上回以選取好的最佳組合 select_features + 這回新選入的一個 feature
            temp_features = select_features + [feature]
            score = Goodness(df,temp_features,y)
            # (目前的特徵組合, 新選進來的特徵, 此特徵組合的 Goodness)
            scores.append((temp_features,feature,score))

        # 依照 Goodness 排序
        scores.sort(key=lambda x: x[2], reverse = True)  
        best_new_score = float(scores[0][2])  
        print("Forward Selection:")
        if(best_new_score > best_score):
            best_score = best_new_score
            select_features = scores[0][0]  # 更新成 Goodness 最優的 subset
            if scores[0][1] in remaining_features:
                remaining_features.remove(scores[0][1])  # 移除新選特徵
            print(f"Pass{i}: best_feature_subset = {select_features} , Goodness = {best_score}")
            i += 1
        # 此輪中所有 feature_subset 的表現皆不如上一輪，Stop
        else:
            break
    print(f"Final select features: {select_features}, Goodness = {best_score}")

def backward_selection(df, X, y):
    select_features = X    
    best_score = 0.0       
    i = 1
    # 持續檢查到選擇的 feature 只剩下一個
    while(len(select_features) > 1):
        scores = [] 
        for feature in select_features:
            temp_features = select_features.copy()
            # 每次移除一個 feature
            temp_features.remove(feature)
            score = Goodness(df,temp_features,y)
            # (目前的特徵組合, 移除的特徵, 此特徵組合的 Goodness)
            scores.append((temp_features,feature,score))

        scores.sort(key = lambda x: x[2], reverse = True)  
        best_new_score = float(scores[0][2]) 
        print("Backward Selection:")
        if(best_new_score >= best_score):
            best_score = best_new_score
            select_features = scores[0][0]  # 更新成 Goodness 最優的 subset
            if scores[0][1] in select_features:
                select_features.remove(scores[0][1])  # 移除特徵
            print(f"Pass{i}: best_feature_subset = {select_features} , Goodness = {best_score}")
            print(f"remove feature: {scores[0][1]}")
            i += 1
        # 此輪中所有 feature_subset 的表現皆不如上一輪，Stop
        else:
            break
    print(f"Final select features: {select_features}, Goodness = {best_score}")

### Equal Width

In [16]:
def equal_width(df,feature,bin_num):
    # 計算每組 bin 區間
    att_value = df[feature]
    max_value = float(max(att_value))
    min_value = float(min(att_value))
    # 計算每組區間寬度
    width = (max_value - min_value) / bin_num

    bins = []  # 儲存每組區間範圍
    # 計算每組區間數值範圍
    for i in range(bin_num):
        start = min_value + i * width
        end = start + width
        bins.append((start,end)) 
    print(f'{feature} with equal width discretization => width = {width}')
    for i, (start, end) in enumerate(bins):
        if i == bin_num - 1:
            print(f'bin{i + 1}: {start} < x <= {end}')
            break
        if i == 0:
            print(f'bin{i + 1}: {start} <= x <= {end}')
        else:
            print(f'bin{i + 1}: {start} < x <= {end}')
    print("=========================================================")
    # 儲存 discretization 後的值
    bin_result = []
    for value in att_value:
        value = float(value)
        for i, (start, end) in enumerate(bins):
            if i == 0 and value >= start and value <= end:
                bin_result.append(i + 1)
                break
            elif i == bin_num - 1 and value > start and value <= end:
                bin_result.append(i + 1)
                break
            elif value > start and value <= end:
                bin_result.append(i + 1)
                break
    return bin_result

def discretize_equal_width(df, features, bin_num):
    new_df = []
    bin_results = {}
    for feature in features:
        bin_results[feature] = equal_width(df, feature, bin_num)
    bin_results['class'] = df['class']
    return bin_results

# 對原始資料所有連續型變數做離散化並存到新的 dict of list 裡
equal_width_df = discretize_equal_width(df, X, 10)
equal_width_df

RI with equal width discretization => width = 0.0022780000000000022
bin1: 1.51115 <= x <= 1.513428
bin2: 1.513428 < x <= 1.515706
bin3: 1.515706 < x <= 1.517984
bin4: 1.517984 < x <= 1.520262
bin5: 1.520262 < x <= 1.52254
bin6: 1.52254 < x <= 1.524818
bin7: 1.524818 < x <= 1.527096
bin8: 1.527096 < x <= 1.529374
bin9: 1.529374 < x <= 1.531652
bin10: 1.531652 < x <= 1.53393
Na with equal width discretization => width = 0.6649999999999998
bin1: 10.73 <= x <= 11.395
bin2: 11.395 < x <= 12.059999999999999
bin3: 12.06 < x <= 12.725
bin4: 12.725 < x <= 13.389999999999999
bin5: 13.39 < x <= 14.055
bin6: 14.055 < x <= 14.719999999999999
bin7: 14.719999999999999 < x <= 15.384999999999998
bin8: 15.384999999999998 < x <= 16.049999999999997
bin9: 16.049999999999997 < x <= 16.714999999999996
bin10: 16.715 < x <= 17.38
Mg with equal width discretization => width = 0.449
bin1: 0.0 <= x <= 0.449
bin2: 0.449 < x <= 0.898
bin3: 0.898 < x <= 1.347
bin4: 1.347 < x <= 1.796
bin5: 1.796 < x <= 2.245
bin6: 2

{'RI': [5,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  5,
  4,
  3,
  3,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  4,
  3,
  5,
  5,
  3,
  3,
  3,
  5,
  3,
  4,
  4,
  7,
  5,
  4,
  6,
  4,
  4,
  4,
  3,
  3,
  1,
  4,
  3,
  3,
  4,
  4,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  6,
  3,
  4,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  3,
  2,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  3,
  4,
  4,
  3,
  3,
  4,
  3,
  3,
  4,
  8,
  6,
  6,
  9,
  10,
  5,
  4,
  7,
  8,
  8,
  4,
  4,
  4,
  4,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  5,
  4,
  3,
  5,
  5,
  4,
  5,
  7,
  4,
  4,
  4,
  3,
  4,
  3,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  4,
  3,
  3,
  3,
  3,
  3,
  5,
  3,
  3,
  3,
  3,
  3,
  5,
  3,
  3,
  4,
  4,
  5,
  2,
  4,
  5,
  5,
  4,
  3,
  4,
  6,
  1,
  1,
  5,
  5,
  5,
  4,
  4,
  4,
  4,
  1,
  4,
  4,
  4,
  1,
  1,
  4,
  6,
  5,
  6,
  3,
  3,
  3,
  3,
  3,
  2,
  2,
  3,
  2

In [17]:
# 對所有特徵離散化後的資料集做 forward selection、backward selection
forward_selection(equal_width_df, X, 'class')
print("=================================================================================================")
backward_selection(equal_width_df, X, 'class')

Forward Selection:
Pass1: best_feature_subset = ['Ba'] , Goodness = 0.3042912813744425
Forward Selection:
Pass2: best_feature_subset = ['Ba', 'Mg'] , Goodness = 0.39168312637244257
Forward Selection:
Pass3: best_feature_subset = ['Ba', 'Mg', 'Na'] , Goodness = 0.40668726201145056
Forward Selection:
Pass4: best_feature_subset = ['Ba', 'Mg', 'Na', 'Al'] , Goodness = 0.4139089508212203
Forward Selection:
Pass5: best_feature_subset = ['Ba', 'Mg', 'Na', 'Al', 'Ca'] , Goodness = 0.41877684363493245
Forward Selection:
Final select features: ['Ba', 'Mg', 'Na', 'Al', 'Ca'], Goodness = 0.41877684363493245
Backward Selection:
Pass1: best_feature_subset = ['Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe'] , Goodness = 0.39904232199605344
remove feature: RI
Backward Selection:
Pass2: best_feature_subset = ['Na', 'Mg', 'Al', 'K', 'Ca', 'Ba', 'Fe'] , Goodness = 0.40820177751439696
remove feature: Si
Backward Selection:
Pass3: best_feature_subset = ['Na', 'Mg', 'Al', 'K', 'Ca', 'Ba'] , Goodness = 0.41495

### Equal Frequency

In [18]:
def equal_frequency(df,feature,bin_num):
    # 先記錄每個 instance 的原始索引及 value，len(df[feature]) 為資料總筆數
    att_value = [(i, float(df[feature][i])) for i in range(len(df[feature]))]
    # 以 attribute value 的值排序
    att_value.sort(key = lambda x : x[1])
    # 計算每個 bin 應該包含的 instances 數量
    frequency = len(att_value) // bin_num
    bins = [] 
    start = att_value[0][1]
    bin_index = 1    # 記錄目前 bin 
    cur_bin_cnt = 0  # 記錄目前 bin 所分配到的 value 個數
    # att_value 已排序過，故會由小到大遍歷，org_index 是紀錄該筆 instance 在未排序前的位置
    for cur,(org_index,value) in enumerate(att_value):
        # 目前 bin 的 value 數量已滿足一個 bin 所應該分配到的 frequency
        if cur_bin_cnt >= frequency and bin_index <= bin_num:
            # 且當前 value 不等於前一個 value 值
            if cur < len(att_value) and value != att_value[cur - 1][1]:
                # 若已計算到最後一個 bin
                if bin_index == bin_num:
                    bins.append((start,att_value[-1][1]))  # end 即為最後一筆 instance(最大值)
                    break
                end = value  # 該 bin 的區間最大值(不包含)
                bins.append((start,end))

                # 切換到下一個 bin
                bin_index += 1
                cur_bin_cnt = 0
                start = att_value[cur][1]  # att_value[i][1] 為下個 bin 的起點
        # 該 bin 裡的 instances 個數加一
        cur_bin_cnt += 1

    # 上述設定在切換下個 bin 時才將 (start,end) 進 bins
    # 有可能迴圈結束，最後一個 bin 的值個數不足一個 frequency，不會切換 bin，因此需要額外判斷防止最後一個 bin 消失
    if len(bins) < bin_num:
        end = att_value[-1][1]   # end 為最後一個元素(最大值)
        bins.append((start, end))

    print(f'{feature} with equal frequency discretization:')
    for i, (start, end) in enumerate(bins):
        if i == 0:
            print(f'bin{i + 1}: {start} <= x <= {end}')
        else:
            print(f'bin{i + 1}: {start} < x <= {end}')
    print("=========================================================")
    # 對 df 做離散化並將新值存到一個 dict of list
    org_value = df[feature]
    bin_result = []
    for value in org_value:
        value = float(value)
        for i, (start, end) in enumerate(bins):
            if i == 0 and value >= start and value <= end:
                bin_result.append(i + 1)
                break
            elif value > start and value <= end:
                bin_result.append(i + 1)
                break
    return bin_result


def discretize_equal_frequency(df, features, bin_num):
    new_df = []
    # 對每個 feature 做離散化，回傳 bin 結果(dict of list）
    bin_results = {}
    for feature in features:
        bin_results[feature] = equal_frequency(df, feature, bin_num)
    bin_results['class'] = df['class']
    return bin_results

equal_frequency_df = discretize_equal_frequency(df,X,10)

RI with equal frequency discretization:
bin1: 1.51115 <= x <= 1.51592
bin2: 1.51592 < x <= 1.51631
bin3: 1.51631 < x <= 1.5167
bin4: 1.5167 < x <= 1.51735
bin5: 1.51735 < x <= 1.51768
bin6: 1.51768 < x <= 1.51811
bin7: 1.51811 < x <= 1.5186
bin8: 1.5186 < x <= 1.51994
bin9: 1.51994 < x <= 1.52196
bin10: 1.52196 < x <= 1.53393
Na with equal frequency discretization:
bin1: 10.73 <= x <= 12.68
bin2: 12.68 < x <= 12.86
bin3: 12.86 < x <= 13.01
bin4: 13.01 < x <= 13.21
bin5: 13.21 < x <= 13.34
bin6: 13.34 < x <= 13.5
bin7: 13.5 < x <= 13.75
bin8: 13.75 < x <= 14.15
bin9: 14.15 < x <= 14.7
bin10: 14.7 < x <= 17.38
Mg with equal frequency discretization:
bin1: 0.0 <= x <= 0.33
bin2: 0.33 < x <= 2.76
bin3: 2.76 < x <= 3.37
bin4: 3.37 < x <= 3.49
bin5: 3.49 < x <= 3.55
bin6: 3.55 < x <= 3.6
bin7: 3.6 < x <= 3.67
bin8: 3.67 < x <= 3.83
bin9: 3.83 < x <= 4.49
Al with equal frequency discretization:
bin1: 0.29 <= x <= 0.87
bin2: 0.87 < x <= 1.15
bin3: 1.15 < x <= 1.24
bin4: 1.24 < x <= 1.31
bin5: 

In [20]:
# 對所有特徵離散化後的資料集做 forward selection、backward selection
forward_selection(equal_frequency_df, X, 'class')
print("=================================================================================================")
backward_selection(equal_frequency_df, X, 'class')

Forward Selection:
Pass1: best_feature_subset = ['Mg'] , Goodness = 0.24095902540972278
Forward Selection:
Pass2: best_feature_subset = ['Mg', 'Ba'] , Goodness = 0.31902345088124423
Forward Selection:
Pass3: best_feature_subset = ['Mg', 'Ba', 'Al'] , Goodness = 0.35549303947336497
Forward Selection:
Pass4: best_feature_subset = ['Mg', 'Ba', 'Al', 'RI'] , Goodness = 0.3752863615702465
Forward Selection:
Pass5: best_feature_subset = ['Mg', 'Ba', 'Al', 'RI', 'K'] , Goodness = 0.3877323986697869
Forward Selection:
Pass6: best_feature_subset = ['Mg', 'Ba', 'Al', 'RI', 'K', 'Na'] , Goodness = 0.3907283561012959
Forward Selection:
Final select features: ['Mg', 'Ba', 'Al', 'RI', 'K', 'Na'], Goodness = 0.3907283561012959
Backward Selection:
Pass1: best_feature_subset = ['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba'] , Goodness = 0.3827512165073733
remove feature: Fe
Backward Selection:
Pass2: best_feature_subset = ['RI', 'Na', 'Mg', 'Al', 'K', 'Ca', 'Ba'] , Goodness = 0.38704071339993934
remove

### Entropy Based

In [354]:
def Ent(df,class_label):
    class_value = df[class_label]
    cnt = Counter(class_value)
    prob = [count / len(class_value) for i,count in cnt.items()]
    Ent = -sum(p * log2(p) for p in prob) 
    return Ent

# cut_index 為資料被分割成兩個 subset 時，「右側區間的起始 index」，此函式計算切割後的資訊增益
def info_gain(df,cut_index,feature,class_label):
    # 將原始資料 feature 和 class 的對應資料與索引組合起來，(索引 , feature 值 , 類別值)
    new_df = [(i,float(df[feature][i]), df[class_label][i]) for i in range(len(df[feature]))]
    feature_class = sorted(new_df,key = lambda x : x[1])  # x[1] 即為 feature 值
    # 取得排序後 feature 值的原始 index 
    sorted_index = [row[0] for row in feature_class]
    left_index = sorted_index[:cut_index]
    right_index = sorted_index[cut_index:]
    total = len(df[class_label])
    # 切左右兩側，從上述得到的左右區間包含的 instances 的原始索引來分配
    left = {class_label : [df[class_label][i] for i in left_index]}
    right = {class_label : [df[class_label][i] for i in right_index]}
    Ent_cut = len(left[class_label]) / total * Ent(left,class_label) + len(right[class_label]) / total * Ent(right,class_label)

    info_gain = Ent(df,class_label) - Ent_cut
    return info_gain

# 找 feature 的最佳切點
def find_cut_point(df, feature, class_label):
    best_info_gain = -1
    best_cut_value = None  # 最佳切點的 instance 值
    # 將 feature 和 class 的對應資料與索引組合起來，(索引 , feature 值 , 類別值)
    feature_with_class = [(i, float(df[feature][i]),df[class_label][i]) for i in range(len(df[class_label]))]
    sorted_data = sorted(feature_with_class,key = lambda x : x[1])  # x[1] 為 feature 值
    for i in range(1,len(sorted_data)):
        # 相鄰兩樣本類別值不一樣，其平均值可以當切點，且 i 開始為右區間
        if sorted_data[i][2] != sorted_data[i - 1][2]:
            cur_info_gain = info_gain(df,i,feature,class_label)
            if cur_info_gain > best_info_gain:
                best_info_gain = cur_info_gain
                best_cut_value = (sorted_data[i][1] + sorted_data[i - 1][1]) / 2
    return best_info_gain , best_cut_value

# 對整個 df 的 feature 欄位做 entropy_base 切割，找所有切點
def split(df,feature,class_label,cut_points):
    best_info_gain,best_cut_value = find_cut_point(df,feature,class_label)

    if (len(df[class_label]) <= 1 or best_cut_value is None or
        len(set(df[class_label])) == 1 or len(set(df[feature])) == 1):
        return
    
    # 創造兩個 dict of list 儲存切割後的 feature 與 class 欄位
    left_set = {feature : [],class_label : []}
    right_set = {feature : [],class_label : []}
    for i in range(len(df[feature])):
        
        value = float(df[feature][i])
        if value <= best_cut_value:
            left_set[feature].append(value)
            left_set[class_label].append(df[class_label][i])
        else:
            right_set[feature].append(value)
            right_set[class_label].append(df[class_label][i])
    if (len(left_set[feature]) == 0 or len(right_set[feature]) == 0 or
        len(left_set[feature]) == len(df[feature]) or len(right_set[feature]) == len(df[feature])):
        return
    # 計算 MDLPC criterion 的 threshold

    # 計算初始區間、切割後的左右區間個包含的 class 種類數量
    k = len(set(df[class_label]))
    k1 = len(set(left_set[class_label]))
    k2 = len(set(right_set[class_label]))
    N = len(df[feature])
    delta = log2(3 ** k - 2) - ((k * Ent(df,class_label) 
        - k1 * Ent(left_set,class_label)
        - k2 * Ent(right_set,class_label)))
    threshold = log2(N - 1) / N + delta / N

    if best_info_gain <= threshold :
        return
    
    cut_points.append(best_cut_value)
    split(left_set,feature,class_label,cut_points)
    split(right_set,feature,class_label,cut_points)
    return cut_points


In [355]:
def discretize_entropy_base(df,features,class_label):
    bin_results = {}
    for feature in features:
        cut_points = []
        cut_points = split(df,feature,"class",cut_points)
        if cut_points == None:
            ent_base_res = [1] * len(df[feature])
            bin_results[feature] = ent_base_res
            continue
        cut_points = sorted(cut_points)
        print(f"{feature}:{cut_points}")
        org_value = df[feature]
        ent_base_res = []
        for value in org_value:
            value = float(value)
            for i,cut_point in enumerate(cut_points):
                if len(cut_points) == 1:
                    if value <= cut_point:
                        ent_base_res.append(i + 1)
                    else:
                        ent_base_res.append(i + 2)
                else:
                    if i == 0 and value <= cut_point:
                        ent_base_res.append(i + 1)
                    elif i == len(cut_points) - 1 and value > cut_point:
                        ent_base_res.append(i + 1)
                    elif value > cut_point and value <= cut_points[i - 1]:
                        ent_base_res.append(i + 1)
        bin_results[feature] = ent_base_res
    bin_results['class'] = df['class']
    return bin_results

In [356]:
ff = ['RI',"Na",'Mg','Al', 'K', 'Ca', 'Ba', 'Fe','Si']
entropy_base_df = discretize_entropy_base(df,ff,'class')
forward_selection(entropy_base_df, ff, 'class')
backward_selection(entropy_base_df, ff, 'class')

RI:[1.517335, 1.517985]
Na:[14.065]
Mg:[0.0, 2.6950000000000003]
Al:[1.4, 1.76]
K:[0.0, 0.035, 0.055, 0.62, 0.745]
Ca:[7.02, 8.32, 10.17]
Ba:[0.0, 0.335]
Fe:[0.0]
Forward Selection:
Pass1: best_feature_subset = ['Mg'] , Goodness = 0.37040112253047947
Forward Selection:
Pass2: best_feature_subset = ['Mg', 'Ba'] , Goodness = 0.4209779203953104
Forward Selection:
Final select features: ['Mg', 'Ba'], Goodness = 0.4209779203953104
Backward Selection:
Pass1: best_feature_subset = ['Na', 'Mg', 'Al', 'K', 'Ca', 'Ba', 'Fe', 'Si'] , Goodness = 0.40630426699936245
remove feature: RI
Backward Selection:
Pass2: best_feature_subset = ['Na', 'Mg', 'Al', 'K', 'Ca', 'Ba', 'Si'] , Goodness = 0.4168011479585509
remove feature: Fe
Backward Selection:
Pass3: best_feature_subset = ['Na', 'Mg', 'Al', 'Ca', 'Ba', 'Si'] , Goodness = 0.4258437899890787
remove feature: K
Backward Selection:
Pass4: best_feature_subset = ['Na', 'Mg', 'Al', 'Ca', 'Ba'] , Goodness = 0.4258437899890787
remove feature: Si
Backward Sel